In [ ]:
from simnibs import sim_struct, run_simnibs
import nilearn.image as img
import nilearn.plotting as niplot
import os
import numpy as np
from numpy import degrees, arcsin, arctan2, deg2rad, cos, sin

# SimNIBS Hot Tips

Data for this notebook can be found in:

```
/projects/jjeyachandra/simnibs_tutorial/data
```

Feel free to symlink or copy for your own personal use (it's my brain...)

## A Basic Run of SimNIBS

Ripped from their tutorial

In [ ]:
# Initialize session
s = sim_struct.SESSION()
s.fnamehead = "../data/sub-CMHP001.msh"
s.pathfem = "../output"

Check the list of available coils to use

In [ ]:
from simnibs import SIMNIBSDIR
os.listdir(os.path.join(SIMNIBSDIR,'ccd-files'))

In [ ]:
# Set TMS position list
tmslist = s.add_tmslist()
tmslist.fnamecoil = "Magstim_70mm_Fig8.nii.gz"

In [ ]:
# Add a position to the position list
pos = tmslist.add_position()
pos.centre = 'C1'
pos.pos_ydir = 'CP1'

In [ ]:
# Options for mapping
s.map_to_fsavg = True
s.map_to_MNI = True

## Will only work with CAT12
s.map_to_surf = False

In [ ]:
run_simnibs(s)

##  MNI Coordinate Projection

MNI coordinates can be transformed into subject space pretty easily with a SimNIBS command-line tool called `mni2subject_coords`.

Here's an example for left DLPFC (-30, 43, 23)

In [ ]:
!mni2subject_coords -m /projects/jjeyachandra/simnibs_tutorial/data/m2m_sub-CMHP001/ \
    -c "-30" "43" "23" \
    -t nonl

Alternatively this can be done directly using the Python API

In [ ]:
import simnibs.msh.transformations as transformations

In [ ]:
m2m_dir = "../data/m2m_sub-CMHP001/"
coords = [-30, 43, 23]
xfm_res = transformations.warp_coordinates(
    coordinates=coords,m2m_folder=m2m_dir,
    transformation_direction='mni2subject')
print(xfm_res)

The coordinates that we used were brain coordinates, so these need to be projected onto the subject scalp. 

In addition, even _MNI TMS Coordinates_ need to be projected onto the scalp. Otherwise the imperfect MNI coordiante may result in a TMS coil embedded within the subject skull or floating above the subject's head.

This can also be achieved via the `transformations` API.

First we need to read in the mesh file which contains the scalp geometry:

In [ ]:
from simnibs.msh import mesh_io

In [ ]:
mesh = mesh_io.read_msh('../data/sub-CMHP001.msh')

In [ ]:
subject_ldlpfc = xfm_res[1]
subject_dlpfc_scalp = transformations.project_on_scalp(
    coords = subject_ldlpfc,
    mesh=mesh,
    distance=1
)
print(subject_dlpfc_scalp)

In [ ]:
t1 = img.load_img('../data/m2m_sub-CMHP001/T1fs_nu_conform.nii.gz')

Original coordinates

In [ ]:
niplot.plot_anat(t1,
                cut_coords=subject_ldlpfc[0])

Projected onto the scalp

In [ ]:
niplot.plot_anat(t1,
                cut_coords=subject_dlpfc_scalp[0])

## Defining an orientation

To define an orientation we use the BrainSight convention in which:

1. 0 degrees is defined as pointing anteriorly
2. 180 degrees is defined as pointing posteriorly
3. (+) rotations rotate counter-clockwise
4. (-) rotations rotate clockwise

If we wanted to:

1. Target left DLPFC
2. Have the coil pointing posteriorly 
3. Angle *away from the midline* at 45 degrees

We'd calculate our final desired "Brainsight angle" as:

1. Start with 180 degrees to point the handle posteriorly
2. Subtract (rotate clockwise) by 45 degrees
3. The BrainSight twist parameter is therefore **155 degrees**

Next, all we need is the scalp normal which will allow us to position the coil such that the coil is flat against the scalp.

**Step 1: Compute the normal of the scalp right under-neath the coil**

In [ ]:
closest_node, node_index = mesh.nodes.find_closest_node([subject_dlpfc_scalp],
                                           return_index=True)
print(closest_node, node_index)

# Extract from list of list
node_index = node_index[0][0]

In [ ]:
node_normal = mesh.nodes_normals(smooth=2).value[node_index]
print(node_normal)

**Step 2: With this we can figure out the rest of the simulation matrix**

The function below takes the following:

1. a normal vector that we computed earlier
2. the "BrainSight twist" parameter in degrees
3. The coil centre position

And returns to you a matsimnibs matrix that can be directly used by simnibs

In [ ]:
def get_matsimnibs(n,twist,centre):
    
    # Compute euler angles
    alpha = arctan2(-n[1],n[2])
    beta = arcsin(n[0])
    gamma = deg2rad(twist)
    
    # Define rotation matrices for XYZ variant of 
    # tait-bryan rotation matrices 
    R_ap = np.array([
        [1, 0, 0],
        [0, cos(alpha), -sin(alpha)],
        [0, sin(alpha), cos(alpha)]
    ])
    
    R_lr = np.array([
        [cos(beta), 0, sin(beta)],
        [0, 1, 0],
        [-sin(beta), 0, cos(beta)]
    ])
    
    R_tw = np.array([
        [cos(gamma), -sin(gamma), 0],
        [sin(gamma), cos(gamma), 0],
        [0, 0, 1]
    ])
    
    # Get full rotation matrix of coil onto scalp plane
    R = R_ap @ R_lr @ R_tw
    
    # Invert normal and X to maintain right-handedness
    R[:3,1] = -R[:3,1]
    R[:3,2] = -R[:3,2]
  
    # Construct matsimnibs
    msn = np.zeros((4,4))
    msn[:3,:3] = R
    msn[:3,3] = centre
    
    return msn

In [ ]:
# Get 45 degree matsimnibs matrix over DLPFC
deg_45_msn = get_matsimnibs(node_normal,twist=155,c=subject_dlpfc_scalp)

In [ ]:
# Now run a simulation with our matsimnibs matrix
s = sim_struct.SESSION()
s.fnamehead = "../data/sub-CMHP001.msh"
s.pathfem = "../output/new_sim"
s.map_to_fsavg = True
s.map_to_MNI = True
tmslist = s.add_tmslist()
tmslist.fnamecoil = "MagVenture_MC_B70.nii.gz"
pos = tmslist.add_position()
pos.matsimnibs = deg_45_msn

In [ ]:
run_simnibs(s)